## TODO 
First i want to try COT , then i will try to use COT + gem 2.5 flash 
after that if that does not increase the accuracy i wil try preprocessing the image first.
if that does not work i will try and and have it in a graph that would have VLM first then answering.


In [ ]:
# Load environment variables from .env file
import os, getpass
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv
import pandas as pd
import os
import PIL.Image
from tqdm import tqdm
from dotenv import load_dotenv
from pydantic_ai import Agent, RunContext,ImageUrl,BinaryContent
load_dotenv()




In [ ]:
agent = Agent('gemini-2.5-flash-lite-preview-09-2025')

@agent.system_prompt
def context():

    return """
    
    You are an expert mathematician. Analyze the provided image and question to arrive at a solution. Follow these steps precisely in your response:
    Step 1: Diagram Deconstruction.
    Identify the type of mathematical diagram (e.g., 'Linear inequality graph,' 'Venn diagram,' 'Histogram').
    Extract all key features directly from the diagram. For a graph, list the equations of the lines, intercepts, and labeled points. For a chart, extract the data points. For a network, list the nodes and edges with their weights.
    Step 2: Question Interpretation.
    State the explicit goal of the question. What specific value or statement needs to be produced?
    Step 3: Mathematical Formulation.
    Based on the deconstructed diagram from Step 1 and the question from Step 2, formulate the necessary mathematical equations, inequalities, or logical statements.
    Step 4: Step-by-Step Calculation.
    Solve the formulation from Step 3. Show every step of your calculation.
    Step 5: Final Answer.
    State the final answer clearly, ensuring it directly addresses the question.
    """
with open(r'C:\Users\Adonis\OneDrive\Desktop\DataScience\evaluation\QAs\Soalan maths\form 4\Chapter 10_Q2.png','rb') as f:
        image_data = f.read()

In [ ]:

result = await agent.run([
    'Solve the following question i want you to first analayze the questions then start solving make sure your answer is complete, dont answer in markdown',
    BinaryContent(data=image_data, media_type='image/png'),
])

print(result.output)

In [ ]:
max_pixels = 1000*1000  #Max resolution for images

def resize_image(pil_image):
    org_width, org_height = pil_image.size
    print(f"Original size: {org_width}x{org_height} = {org_width*org_height} pixels")

    # Resize image if too large
    if org_width * org_height > max_pixels:
        scale_factor = (max_pixels / (org_width * org_height)) ** 0.5
        new_width = int(org_width * scale_factor)
        new_height = int(org_height * scale_factor)
        pil_image.thumbnail((new_width, new_height))
        print(f"Resized to: {pil_image.size[0]}x{pil_image.size[1]} = {pil_image.size[0]*pil_image.size[1]} pixels")
    else:
        print("No resize needed")
    
    return pil_image

In [ ]:
# Load environment variables (assuming already loaded in previous cells)
# client should already be initialized from your previous cells

def get_llm_answer_from_image(img_path):
    """
    Read an image containing a math question and get the LLM to answer it
    """
    # Load and resize image
    pil_image = PIL.Image.open(img_path)
    pil_image = resize_image(pil_image)
    model = "gemini-2.0-flash"


    prompt = ["""You are a math expert. 
    Look at this image which contains a math question.
    Read the question from the image and solve it step by step.
    Show your working clearly.
    Don't use markdown.
    """, pil_image]
    
    try:
        response = client.models.generate_content(
            model=model,
            contents=prompt
        )
        
        answer = response.text
        
        # Track token usage if available
        if hasattr(response, 'usage_metadata') and response.usage_metadata:
            tokens = response.usage_metadata.total_token_count
            return answer, tokens, model
        
        return answer, 0
    except Exception as e:
        return f"Error: {e}", 0

def process_test_images(input_csv, img_folder, output_csv):
    """
    Process test images and generate answers
    
    Expected CSV columns:
    - image_filename: Name of the image file (e.g., "question1.jpg")
    - ground_truth: The correct answer
    - marking_scheme: The marking criteria
    """
    # Read input CSV
    print(f"Reading CSV from {input_csv}...")
    df = pd.read_csv(input_csv)

    
    # Validate required columns
    required_cols = ['image_filename', 'ground_truth', 'marking_scheme']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    # Initialize answer columns
    df['llm_answer'] = ""
    df['tokens_used'] = 0
    df['status'] = ""
    df['model'] = ""
    
    total_tokens = 0
    
    # Process each image
    print(f"\nProcessing {len(df)} images...")
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        img_filename = row['image_filename']
        img_path = os.path.join(img_folder, img_filename)
        
        # Check if image exists
        if not os.path.exists(img_path):
            print(f"\nWarning: Image not found: {img_path}")
            df.at[idx, 'llm_answer'] = "Image not found"
            df.at[idx, 'status'] = "ERROR"
            continue
        
        print(f"\nProcessing: {img_filename}")
        
        # Get LLM answer
        answer, tokens, model = get_llm_answer_from_image(img_path)
        
        df.at[idx, 'llm_answer'] = answer
        df.at[idx, 'tokens_used'] = tokens
        df.at[idx, 'status'] = "SUCCESS" if not answer.startswith("Error") else "ERROR"
        df.at[idx, 'model'] = model
        
        total_tokens += tokens
        print(f"Tokens used: {tokens} | Total so far: {total_tokens}")
    
    # Save output CSV
    print(f"\nSaving results to {output_csv}...")
    df.to_csv(output_csv, index=False)
    
    # Print summary
    print("\n" + "="*60)
    print("TEST SUMMARY")
    print("="*60)
    print(f"Total images processed: {len(df)}")
    print(f"Successful: {len(df[df['status'] == 'SUCCESS'])}")
    print(f"Errors: {len(df[df['status'] == 'ERROR'])}")
    print(f"Total tokens used: {total_tokens}")
    print(f"Results saved to: {output_csv}")
    print("="*60)
    
    return df

# Run the test
input_csv = "test_questions_mathform5.csv"      # CSV with: image_filename, ground_truth, marking_scheme
img_folder = "Soalan maths\\form 5"                     # Folder containing the question images
output_csv = "test_results_f5.csv"        # Output file with answers

results_df = process_test_images(input_csv, img_folder, output_csv)

# Display first few results
print("\nFirst 3 results:")
print(results_df[['image_filename', 'ground_truth', 'llm_answer', 'status']].head(3))

In [ ]:
# Load environment variables (assuming already loaded in previous cells)
# client should already be initialized from your previous cells
# or "gpt-4-vision-preview" or "gpt-4o-mini"
def get_openai_answer_from_image(img_path, model):
    """
    Read an image containing a math question and get OpenAI to answer it
    """
    import base64
    
    # Load and resize image
    pil_image = PIL.Image.open(img_path)
    pil_image = resize_image(pil_image)
    
    # Convert to base64
    import io
    buffered = io.BytesIO()
    pil_image.save(buffered, format="PNG")
    base64_image = base64.b64encode(buffered.getvalue()).decode('utf-8')
    
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": """You are a math expert. 
Look at this image which contains a math question.
Read the question from the image and solve it step by step.
Show your working clearly.
Don't use markdown."""
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{base64_image}"
                    }
                }
            ]
        }
    ]
    
    try:
        response = openai_client.chat.completions.create(
            model=model,  # or "gpt-4-vision-preview" or "gpt-4o-mini"
            messages=messages
        )
        
        answer = response.choices[0].message.content
        tokens = response.usage.total_tokens
        
        return answer, tokens, model
    except Exception as e:
        return f"Error: {e}", 0

def process_test_images(input_csv, img_folder, output_csv, model):
    """
    Process test images and generate answers
    
    Expected CSV columns:
    - image_filename: Name of the image file (e.g., "question1.jpg")
    - ground_truth: The correct answer
    - marking_scheme: The marking criteria
    """
    # Read input CSV
    print(f"Reading CSV from {input_csv}...")
    df = pd.read_csv(input_csv)

    
    # Validate required columns
    required_cols = ['image_filename', 'ground_truth', 'marking_scheme']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")
    
    # Initialize answer columns
    df['llm_answer'] = ""
    df['tokens_used'] = 0
    df['status'] = ""
    df['model'] = ""
    
    total_tokens = 0
    
    # Process each image
    print(f"\nProcessing {len(df)} images...")
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        img_filename = row['image_filename']
        img_path = os.path.join(img_folder, img_filename)
        
        # Check if image exists
        if not os.path.exists(img_path):
            print(f"\nWarning: Image not found: {img_path}")
            df.at[idx, 'llm_answer'] = "Image not found"
            df.at[idx, 'status'] = "ERROR"
            continue
        
        print(f"\nProcessing: {img_filename}")
        
        # Get LLM answer
        answer, tokens, model = get_openai_answer_from_image(img_path, model)
        
        df.at[idx, 'llm_answer'] = answer
        df.at[idx, 'tokens_used'] = tokens
        df.at[idx, 'status'] = "SUCCESS" if not answer.startswith("Error") else "ERROR"
        df.at[idx, 'model'] = model
        
        total_tokens += tokens
        print(f"Tokens used: {tokens} | Total so far: {total_tokens}")
    
    # Save output CSV
    print(f"\nSaving results to {output_csv}...")
    df.to_csv(output_csv, index=False)
    
    # Print summary
    print("\n" + "="*60)
    print("TEST SUMMARY")
    print("="*60)
    print(f"Total images processed: {len(df)}")
    print(f"Successful: {len(df[df['status'] == 'SUCCESS'])}")
    print(f"Errors: {len(df[df['status'] == 'ERROR'])}")
    print(f"Total tokens used: {total_tokens}")
    print(f"Results saved to: {output_csv}")
    print("="*60)
    
    return df

# Run the test
input_csv = "test_questions_mathform5.csv"      # CSV with: image_filename, ground_truth, marking_scheme
img_folder = "Soalan maths\\form 5"                     # Folder containing the question images
output_csv = "test_results_f5_gpt5mini.csv"        # Output file with answers
model="gpt-5-mini"

results_df = process_test_images(input_csv, img_folder, output_csv, model)

# Display first few results
print("\nFirst 3 results:")
print(results_df[['image_filename', 'ground_truth', 'llm_answer', 'status']].head(3))